# Short course introduction to Machine Learning: Classification
### March 14, 2017

Prepared by [Heiko Strathmann](http://www.herrstrathmann.de/). Loosely based on previous versions by: Dino Sejdinovic, John Shawe-Taylor, Tom Diethe

## Abstract
Using a series of examples, in this exercise session you will familiarise yourselves with the naive Bayes classifier and support vector machines.
This is an interactive IPython notebook.
It runs on a remote server to which you connect via the web-browser.
The original version of the notebook is available [here](https://github.com/karlnapf/machine_learning_course).

NOTE: The notebook makes use of the Shogun Machine Learning Toolbox, http://shogun.ml. If you want to run it locally you need to install Shogun.
You could also run it in the Shogun cloud at http://cloud.shogun.ml

## Importing necessary libraries
We here use [numpy](http://www.numpy.org/), [matplotlib](http://matplotlib.org/) and [Shogun](http://shogun.ml).
Make sure to check the documentation available online.
If you are coming from Matlab, [this](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html) might be helpful.

In [ ]:
import numpy as np
import modshogun as sg

import matplotlib.pyplot as plt
%matplotlib inline

## Naive Bayes classifier

Assume that we have training examples $S=\left\{ (\mathbf{x}^{(i)},y^{(i)})\right\} _{i=1}^{m}$,
where each $\mathbf{x}^{i}=\left(x_{1}^{(i)},\ldots,x_{D}^{(i)}\right)$
is a $D$-dimensional vector and $y^{(i)}$ is the corresponding label.
For a new data point $\mathbf{x}_{tst}$, we wish to predict its label
$y_{tst}$ using the Bayes theorem:

\begin{eqnarray*}
y_{tst} & = & \arg\max_{y}P(y|\mathbf{x}_{tst})\\
 & = & \arg\max_{y}\frac{P(\mathbf{x}_{tst}|y)P(y)}{P(\mathbf{x}_{tst})}\\
 & = & \arg\max_{y}P(\mathbf{x}_{tst}|y)P(y),
\end{eqnarray*}
since denominator does not depend on $y$. However, this requires
estimation of a high-dimensional probability distribution $P(\mathbf{x}|y)$,
which is impossible in most interesting cases. Naive Bayes makes a
strong (*naive*) independence assumption on this probability
distribution, i.e., that
\begin{eqnarray*}
P(\mathbf{x}|y) & = & \prod_{j=1}^{D}P(x_{j}|y),
\end{eqnarray*}
i.e., individual components of $\mathbf{x}$ are conditionally independent
given its label $y$. The classifier then proceeds by estimating $D$
one dimensional distributions $P(x_{j}|y)$, which is a much easier
task. For example, when variables $x_{j}$ are binary, estimation
of $P(x_{j}|y)$, can be expressed through:
\begin{eqnarray*}
p_{jk} & = & P(x_{j}=1|y=k),\quad\pi_{k}=P(y=k),
\end{eqnarray*}
with maximum likelihood (ML) estimates given by:
\begin{eqnarray}
\hat{p}_{jk} & = & \frac{\#\left\{ (\mathbf{x},y)\in S\,:\;x_{j}=1,y=k\right\} }{\sum_{l=1}^{D}\#\left\{ (\mathbf{x},y)\in S\,:\;x_{l}=1,y=k\right\} },\\
\hat{\pi}_{k} & = & \frac{\#\left\{ (\mathbf{x},y)\in S\,:\;y=k\right\} }{m}.
\end{eqnarray}


We will use a real-world dataset of text extracted from Yahoo! pages which has been used in previous studies, 

`A. Vinokourov, D. R. Hardoon and J. Shawe-Taylor (2003), Learning the Semantics of Multimedia Content with Application to Web Image Retrieval and Classification. 4th International Symposium on Independent Component Analysis (ICA 2003), Nara, Japan.`

and

`T. Joachims (2002). Learning to Classify Text Using Support Vector Machines.`

We will construct a classifier that predicts whether a given document belongs to 'Sport' or 'Aviation' category.
As working with text itself is unwieldy, we use the so called 'bag of words' representation for the documents.
We construct a dictionary, i.e. the union of all $D$ words occurring in any of the texts.
Then each document can be represented by a $D$-dimensional vector encoding the number of times that each of the $D$ words occurs in that document.
(Note that the 'bag of words' representation is clearly very crude, since it does not take into account the order of the words).
This leads to a notion of similarity (kernel) between two text documents, as an inner product between these 'bag of words' representations, given by:
\begin{eqnarray*}
 &  & k(\texttt{text 1},\texttt{text 2})=
 &  & \sum_{{\texttt{word}}\ \in\ {\texttt{dictionary}}}(\#{\texttt{occ. of word} \in \texttt{text 1}})\times(\#{\texttt{occ. of word} \in \texttt{text 2}}).
\end{eqnarray*}


## The dataset

We begin by loading the two sets of documents and exploring their size and content.

In [ ]:
with open("docs_sport.txt", 'r') as f:
    docs_sport = f.readlines()

with open("docs_aviation.txt", 'r') as f:
    docs_aviation = f.readlines()

In [ ]:
print type(docs_sport), len(docs_sport)
print type(docs_aviation), len(docs_sport)

In [ ]:
print docs_sport[:5]

In [ ]:
print docs_aviation[:2]

We now write a function that 'stacks' the two sets of documents on top of each other, forming the $x$ of the dataset.
We will use this function throughout the exercise.
In order to classify the documents, we need to generate a vector of labels $y$ for them.
This is simply a vector $y$ of which each component corresponds to exactly one document.
We here set them to be either $+1$ or $-1$, based on which document class it corresponds to.

In [ ]:
def concat_two_class_data(Xs, Ys):
    """
    Turns a set of Xs and Ys into a classification problem X with binarly labels y.
    """
    
    assert type(Xs) == type(Ys)
    
    if type(Xs) == np.ndarray:
        # deal with plain vectors and matrices
        if Xs.ndim==2 and Ys.ndim==2:
            X = np.vstack([Xs, Ys])
        elif Xs.ndim==1 and Ys.ndim==1:
            X = np.hstack([Xs, Ys])
    elif type(Xs) == list:
        # lists can be appended
        X = Xs + Ys
    else:
        raise TypeError("Unknown data type")
        
    y = np.zeros(len(X))
    y[:len(Xs)]=1.
    y[len(Xs):]=-1.
    
    return X, y

In [ ]:
docs, y = concat_two_class_data(docs_sport, docs_aviation)
print len(docs)
print y.shape
print np.unique(y)

We now turn the documents into a the bag-of-words representation mentioned above.
For that, we first generate a single big string in which we concatenate all documents, using [join](https://www.tutorialspoint.com/python/string_join.htm). We then [split](https://www.tutorialspoint.com/python/string_split.htm) this big string into individual words, each of which we [strip](https://www.tutorialspoint.com/python/string_strip.htm) to remove any spaces at beginning or end.
Finally, we extract a list of unique words.

In [ ]:
joined_docs = " ".join(docs)
all_words = joined_docs.split(" ")
all_words = [word.strip(" ") for word in all_words]
all_words = np.unique(all_words)
print all_words[2350:2360]

In order to turn a document into a feature vector, we now simply for each document need to [count](https://www.tutorialspoint.com/python/string_count.htm) for each word how many times it appears in the doc.
This produces a binary vector whose length is equal to the number of unique words in all documents.
Note that we normalise by the document length to avoid larger documents to be over-weighted.
Also note: this will take a moment.

In [ ]:
X = np.zeros((len(docs), len(all_words)))
for i,doc in enumerate(docs):
    X[i] = np.array([doc.count(w) for w in all_words], dtype=np.float64)
    X[i] /= len(doc.split(" "))

In [ ]:
print X.shape
print X[0]
print np.sum(X[1])

We next wish to randomly split the data into training and testing parts. For that, we write another function that we will re-use later.

In [ ]:
def split_train_test(X, y, train_ratio = 0.1):
    """
    Takes 2d array X and 1d array y and (randomly) splits into train and test parts, based on the provided ratio.
    """
    assert len(X) == len(y)
    assert train_ratio>0 and train_ratio <1
    
    num_train = np.int(np.round(len(X))*train_ratio)
    inds = np.random.permutation(len(X))

    inds_train = inds[:num_train]
    inds_test = inds[num_train:]
    
    X_train = X[inds_train]
    y_train = y[inds_train]
    X_test = X[inds_test]
    y_test = y[inds_test]
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = split_train_test(X,y,train_ratio=0.125)

print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

## Running naive Bayes

We are now ready to apply the naive Bayes classifier to predict the class label of the test documents.
As the naive Bayes classifier for binary vectors is extremely simple, we can just implement it in a few lines.
We provide you with a Python implementation of it.
Try to read the below code and understand how it connects to the Math description above.
It is no problem if you don't understand all the details for now.

In [ ]:
def naive_bayes_binary(X_train,y_train, X_test):
    classes = np.unique(y_train)
    num_classes = len(classes)
    D = X_train.shape[1]
    m = X_train.shape[0]
    
    pie = np.array([np.float64(np.sum(y==k))/m for k in classes])
    
    p = np.zeros((D, num_classes))
    log_posterior_test = np.zeros((len(X_test), num_classes))
    for k,c in enumerate(classes):
        all_docs_class_c = X_train[y==c]
        num_occurences_of_each_feature = np.sum(all_docs_class_c, axis=0)+1
        num_occurences = np.sum(all_docs_class_c)
        p[:,k] = num_occurences_of_each_feature / num_occurences

        log_posterior_test[:,k] = np.dot(X_test,np.log(p[:,k])) + np.log(pie[k]);

    return np.argmax(log_posterior_test, axis=1)   

We run the naive Bayes classifer and predict the labels of the test data. We can then compute the number of mismatches of the 'true' test labels and the predicted test labels.

In [ ]:
y_pred = naive_bayes_binary(X,y, X_test)

print "Predictions", y_pred[:16]
print "True labels", y_test[:16]

y_pred_binary = y_pred*2-1
print "Test accuracy is %.2f" % np.mean(y_pred_binary == y_test)

## Tasks

 * Check how many documents in each of the classes were misclassified
 * Look at the misclassified documents
 * Experiment with different sizes of splits of training and test data, and mark how performance depends on the size of the training data.

## Summary

Naive Bayes is one of the simplest density estimation methods from which we can form one of the standard classification methods in machine learning.
Its fame is partly due to the following properties: 

 * Very easy to program and intuitive 
 * Fast to train and to use as a classifier 
 * Very easy to deal with missing attributes (we did not do that here)
 * Very popular in fields such as computational linguistics/NLP 

As we have seen, naive Bayes can be useful in classification of text documents.
The reason that naive Bayes may be able to classify documents reasonably well in this way is that the conditional independence assumption is not so silly:
if we know people are talking about politics, this
perhaps is almost sufficient information to specify what kinds of other words they will be using - we don't need to know anything else (of course, if you want ultimately a more powerful text classifier, you need to relax this assumption).

## Comparing to SVM

The above accuracy is already quite good. However, the naive Bayes classifier makes some extremely simplifying assumptions about the data. We will develop more intuition on this later. For now, let's see how the mighty support vector machine (SVM) performs on this task. We use an implementation of the open-source [Shogun Machine Learning Toolbox](http://shogun.ml) here as the SVM is not so staight-forward to implement (efficienctly).

In order to use Shogun, we need to pass the data objects to it.
For that, we define another helper function. This does simply copy our training objects to Shogun.
In this notebook, we will use the a prefix `sg_` for all projects that are owned by Shogun, e.g. `sg_y_train` is the Shogun representation of y_train.


If you are interested, have a look at the [API](http://shogun.ml/api) of the classes: `DenseFeatures`, `BinaryLabels`, `MulticlassLabels`, `LibSVM`, `LinearKernel`, `GaussianKernel`, `CrossValidation` and check out usage [examples](http://shogun.ml/examples).
At http://shogun.ml/showroom, you can also find notebooks for classification, regression, support vector machines, and model-selection.

In [ ]:
def pass_to_shogun(X_train, X_test, y_train, y_test, multiclass_labels=False):
    """
    Converts the given objects into Shogun representation
    """
    sg_X_train = sg.RealFeatures(X_train.T)
    sg_X_test = sg.RealFeatures(X_test.T)
    if multiclass_labels:
        sg_y_train = sg.MulticlassLabels((y_train+1)/2.)
        sg_y_test = sg.MulticlassLabels((y_test+1)/2.)
    else:
        sg_y_train = sg.BinaryLabels(y_train)
        sg_y_test = sg.BinaryLabels(y_test)

    return sg_X_train, sg_X_test, sg_y_train, sg_y_test

In [ ]:
sg_X_train, sg_X_test, sg_y_train, sg_y_test = pass_to_shogun(X_train, X_test, y_train, y_test)

C = 50.0
sg_svm = sg.LibSVM(C, sg.LinearKernel(), sg_y_train)
sg_svm.train(sg_X_train);

sg_y_pred = sg_svm.apply(sg_X_test)
y_pred = sg_y_pred.get_labels()

print "Test accuracy is %.2f" % np.mean(y_pred == y_test)

This is a much better result than the naive Bayes classifier. Note that accuracy sometimes is not a good measure of performance. especially for non-symmetric class label ratios. With Shogun, it is easy to use other measures.

In [ ]:
print sg.AccuracyMeasure().evaluate(sg_y_pred, sg_y_test)
print sg.ROCEvaluation().evaluate(sg_y_pred, sg_y_test)
print sg.F1Measure().evaluate(sg_y_pred, sg_y_test)

## When Naive Bayes fails

We will now explore the independence assumption that naive Bayes makes in more detail.
For that, we will consider a continuous case (previously our data was binary vectors, now they are real-valued vectors).

We provide you with a helper function that generates two blos of 2D [normal distributed](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) data.
Try to understand the function, but do not worry if you don't -- we will plot it below.

In [ ]:
def gen_elongated_gaussians(n_x=400, n_y=400,
                            mu_x=np.array([1, -1.0]), mu_y=np.array([-1, 1.0]),
                            inv_eig_vals=np.array([2,0.01]), noise_dim=0):

    n_x=400;
    n_y=400;

    # covariance defined below via eigendecomposition C=U*S*U'

    # U controls the eigenvectors of the covariance matrix
    theta=-np.pi/4;
    U   =  np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta),  np.cos(theta)]])
    s_inv = np.array(inv_eig_vals)          
    Sigma = np.dot(U, np.diag(1.0/s_inv)).dot(U.T)

    # sample from multivariate Gaussian with covariance Sigma
    L = np.linalg.cholesky(Sigma)
    Xs =  L.dot(np.random.randn(2, n_x)).T + mu_x
    Ys =  L.dot(np.random.randn(2, n_y)).T + mu_y

    # add noise_dim dimensions of "noise" 
    # increase to make the problem harder
    if noise_dim>0:
        Xs = np.hstack([Xs, np.random.randn(n_x, noise_dim)])
        Ys = np.hstack([Ys, np.random.randn(n_y, noise_dim)])
        
    return Xs, Ys

In [ ]:
def plot_two_class_data(Xs, Ys, item_strings=["Xs", "Ys"]):
    plt.plot(Xs[:,0], Xs[:,1], 'r.')
    plt.plot(Ys[:,0], Ys[:,1], 'b.')
    plt.legend(item_strings)
    plt.grid(True)
    plt.xlabel(item_strings[0])
    plt.ylabel(item_strings[1])
    plt.axes().set_aspect('equal', 'datalim')

We begin with generating dataset where Gaussian naive Bayes works well, i.e. where the 'independence of the components' assumption holds.

In [ ]:
Xs, Ys = gen_elongated_gaussians(inv_eig_vals=[1., 1.])
plot_two_class_data(Xs, Ys)

As the Gaussian version of naive Bayes is slightly more involved to program, we use Shogun here.

In [ ]:
X,y = concat_two_class_data(Xs, Ys)
X_train, X_test, y_train, y_test = split_train_test(X,y,train_ratio=0.125)
sg_X_train, sg_X_test, sg_y_train, sg_y_test = pass_to_shogun(X_train, X_test, y_train, y_test, multiclass_labels=True)

sg_gnb = sg.GaussianNaiveBayes()
sg_gnb.set_features(sg_X_train)
sg_gnb.set_labels(sg_y_train)

sg_gnb.train();

sg_y_pred = sg_gnb.apply(sg_X_test)

print "Test accuracy:", sg.MulticlassAccuracy().evaluate(sg_y_pred, sg_y_test)

This results looks good.
Here is a function that plots how the predictions on test data look like.

In [ ]:
def plot_two_class_predictions(X_test, y_pred, item_strings=["Xs", "Ys"]):
    y_values = np.unique(y_pred)
    
    X_pos = X_test[y_pred==y_values[0]]
    
    if len(y_values)>1:
        X_neg = X_test[y_pred==y_values[1]]
    
    plt.plot(X_pos[:,0],X_pos[:,1], 'b.')
    if len(y_values)>1:
        plt.plot(X_neg[:,0],X_neg[:,1], 'r.')
    plt.legend(["Predicted %s" % item_strings[0], "Predicted %s" % item_strings[1]])
    plt.grid(True)


In [ ]:
plot_two_class_predictions(X_test, sg_y_pred.get_values()*2-1)

As you see, Gaussian naive Bayes seems to do well on this problem.
This comes from the fact that the two dimensions of the data are indeed independent (i.e. have 'round' point clouds). 

Next, we look at a case where there is correlation in the data.

In [ ]:
Xs, Ys = gen_elongated_gaussians()

plot_two_class_data(Xs, Ys)

X,y = concat_two_class_data(Xs, Ys)
X_train, X_test, y_train, y_test = split_train_test(X,y,train_ratio=0.125)

sg_X_train, sg_X_test, sg_y_train, sg_y_test = pass_to_shogun(X_train, X_test, y_train, y_test, multiclass_labels=True)

sg_gnb = sg.GaussianNaiveBayes()
sg_gnb.set_features(sg_X_train)
sg_gnb.set_labels(sg_y_train)

sg_gnb.train();

sg_y_pred = sg_gnb.apply(sg_X_test)
y_pred = sg_y_pred.get_labels()

plt.figure()
plot_two_class_predictions(X_test, y_pred)
print "Test accuracy:", sg.MulticlassAccuracy().evaluate(sg_y_pred, sg_y_test)

As you can see, Gaussian naive Bayes completely fails on this example.
Re-run it a few times to see how the algorithm tries to predict 'round' point clounds in the test data.

## Task

 * Discuss the independence assumption with the people around you (neighbour, TA), and make sure you get the intuition.
 * Run a SVM on the same dataset, taking inspiration from the the case above. You should get almost perfect accuracy (>95%)
 * Have a look at the function `plot_support_vectors` below.
 Pass the trained svm, and `x_range=[-30,30], y_range=[-30,30]`.
 It will show you the support vectors that span the solution, and a heatmap of the 'distance from the separating hyper-plane'.
 What can you see? Discuss this with your neighbour.

In [ ]:
# run SVM on the correlated dataset

In [ ]:
def plot_support_vectors(sg_svm,X_train,y_train,x_range, y_range):
    res = 100
    x = np.linspace(x_range[0], x_range[1], res)
    y = np.linspace(y_range[0], y_range[1], res)

    xx, yy=np.meshgrid(x, y)
    grid=np.array((np.ravel(xx), np.ravel(yy)))
    sg_out=sg_svm.apply(sg.RealFeatures(grid))
    z=sg_out.get_values().reshape((res, res))

    plt.jet()
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    plt.pcolor(x, y, z)
    plt.contour(x , y, z, linewidths=1, colors='black', hold=True)
    plt.scatter(X_train[:,0],X_train[:,1], c=y_train)
    plt.title("Training data")

    sv = sg_svm.get_support_vectors()
    plt.subplot(122)
    plt.jet()
    c=plt.pcolor(x, y, z)
    plt.contour(x , y, z, linewidths=1, colors='black', hold=True)
    plt.colorbar(c)
    plt.scatter(X_train[sv,0],X_train[sv,1], c=y_train[sv])
    plt.title("Support vectors");

In [ ]:
# plot the support vectors
# plot_support_vectors(sg_svm,X_train,y_train,x_range=[-30,30], y_range=[-30,30])

## Non linearly-separable data

We now consider a simple synthetic data which illustrates the
strength of non-linear kernels.

For that, we use the above Gaussian data script to generate 2D data, where positive examples are a mixture of bivariate gaussians with means $(-1,-1)$ and $(1,1)$ while the negative examples are a mixture of bivariate gaussians with means $(-1,1)$ and $(1,-1)$.
All Gaussian components have the same covariance matrix.

## Task
 * What is the best line that separates the data?

In [ ]:
Xs, Ys = gen_elongated_gaussians(mu_x=np.array([-1, -1.0]), mu_y=np.array([-1, 1.0]), inv_eig_vals=np.array([4,2]))
Xs_, Ys_ = gen_elongated_gaussians(mu_x=np.array([1, 1.0]), mu_y=np.array([1, -1.0]), inv_eig_vals=np.array([4,2]))

Xs = np.vstack([Xs, Xs_])
Ys = np.vstack([Ys, Ys_])
plot_two_class_data(Xs, Ys)

Does the SVM work well on this example?

In [ ]:
X,y = concat_two_class_data(Xs, Ys)
X_train, X_test, y_train, y_test = split_train_test(X,y,train_ratio=0.125)
sg_X_train, sg_X_test, sg_y_train, sg_y_test = pass_to_shogun(X_train, X_test, y_train, y_test)

sg_svm = sg.LibSVM(C, sg.LinearKernel(), sg_y_train)
sg_svm.train(sg_X_train);
sg_svm.train();

sg_y_pred = sg_svm.apply(sg_X_test)
plot_two_class_predictions(X_test, sg_y_pred.get_labels())
print "Test accuracy:", sg.AccuracyMeasure().evaluate(sg_y_pred, sg_y_test)

You can re-run the above code a few times.
You will see that the predicted labels are always separated by a line.
Since there is no line that separates the training data, we get poor accuracy.

To address this problem, we now run SVM with a Gaussian rbf kernel $$k(x,x')=\exp(-\sigma\left\Vert x-x'\right\Vert _{2}^{2})$$
with parameters $\sigma=1$.

A SVM with
a gaussian kernel first maps the data from $\mathbb{R}^{2}$ into a higher dimensional feature space, where the resulting 'features' can be linearly separated, and a maximum-margin hyperplane is fitted in this space.
When mapped back to our original space, the classifier is non-linear.
To get some intuition of how this works, you can have a look [here](http://www.youtube.com/watch?v=3liCbRZPrZA), where a polynomial kernel with a three-dimensional feature space is used.
In general, feature space can have a large and even an infinite number of dimensions.

In [ ]:
sigma = 1.
sg_kernel = sg.GaussianKernel()
sg_kernel.set_width(sigma)

C=1.
sg_svm = sg.LibSVM(C, sg_kernel, sg_y_train)
sg_svm.train(sg_X_train);

sg_y_pred = sg_svm.apply(sg_X_test)
plot_two_class_predictions(X_test, sg_y_pred.get_labels())
print "Test accuracy:", sg.AccuracyMeasure().evaluate(sg_y_pred, sg_y_test)

As you can see, the prediction are now much better.

## Task

 * Experiment with different values of sigma, ranging from very small ($\sigma=2^{-8}$) to very large ($\sigma=2^{8}$). How does the choice affect the test accuracy?
 * Plot the predicted labels for the **training data** for very small $\sigma$. What can you see? 
 * Read on [overfitting](https://en.wikipedia.org/wiki/Overfitting)

## Parameter choice and cross-validation

Let us now do a systematic search for the kernel width $\sigma$.
For that, we generate a sequence of widths to try,  train the SVM on each of them, and monitor performance.
In theory, we then can choose the parameter that worked best.
Note that the test set is typically unavailable for parameter training (you might not even have seen it when you are building your prediction model).
So let's just use the training data to measure performance.

## Task
 * Fill in the loop in the code below to:
  * set the kernel to the current `width`
  * train the svm
  * store the test accuracy in `results[i]`

In [ ]:
widths = 2**np.linspace(-10, 10, 20)

results = np.zeros(len(widths))
for i,width in enumerate(widths):
    # TASK: fill in the loop
    pass # <- delete this

plt.plot(np.log2(widths), results, 'b-')
plt.xlabel(r"$\log_2 (\sigma)$")
plt.ylabel("Accuracy")
plt.grid(True)

plt.plot(np.log2(widths[np.argmax(results)]), np.max(results), 'r*', markersize=15);

If you did everything right, there should be a plot.
From the plot it seems like we can get almost perfect performance by choosing a very small $\sigma$.
That seems great, let's train the SVM with the best $\sigma$.
Now that we have trained the model, we can imagine somebody gives us test data, and we apply our almost perfect SVM to it.

In [ ]:
best_width = widths[np.argmax(results)]
sg_kernel.set_width(best_width)
sg_svm.train()
sg_y_pred = sg_svm.apply(sg_X_test)


plot_two_class_predictions(X_test, sg_y_pred.get_labels())
print "Test accuracy of best width of %.2f is %.2f" % (best_width, sg.AccuracyMeasure().evaluate(sg_y_pred, sg_y_test))

## Task

 * What happened? Training accuracy was almost 100%, but test accuracy is almost as bad as it can get: 50%

Two extremely important aspects of machine learning are:

 * Never ever tune the parameters of your algorithm on training data -- you will overfit. 
 * If you use test data to tune parameters of your algoritm, it is **not** test data anymore. If you want to report test performance, you need to get more data.
 
With that in mind, we now use cross-validation to tune the parameters.
Below, we show you how to compute the cross-validation accuracy.
This quantify behaves similar to the test accuracy, but you compute it from only looking at the training data.

In [ ]:
num_folds = 5
sg_split = sg.StratifiedCrossValidationSplitting(sg_y_train, num_folds)
sg_xval = sg.CrossValidation(sg_svm, sg_X_train, sg_y_train,
                          sg_split, sg.AccuracyMeasure())

sg_xval.set_num_runs(5)
sg_xval.get_global_parallel().set_num_threads(2)

sigma = 2**-10
sg_kernel.set_width(sigma)
print "X-validation accuracy for sigma=%.2f is %.2f" % (sigma, sg.CrossValidationResult.obtain_from_generic(sg_xval.evaluate()).mean)

sigma = 1
sg_kernel.set_width(sigma)
print "X-validation accuracy for sigma=%.2f is %.2f" % (sigma, sg.CrossValidationResult.obtain_from_generic(sg_xval.evaluate()).mean)


As you can see, cross-validation is able to detect bad parameters (the ones that give you high training accuracy but low test accuracy.

Here is how it works:
 * It divides the training set in $k$ disjoint sets.
 * Each of these $k$ sets of samples is once lifted out as the validation set, and the remaining $k-1$ sets are used for training.
 * As a result, we get $k$ validation scores.  The average of these scores is used as a good estimate of the test set performance.

## Task

 * Can you implement cross-validation yourself? Have a look at the function `split_train_test` from above on how to split data.
 * Hint: you could actually start by using the `split_train_test` directly.
 The subsets would not be disjoint in this case, but you should still be able to get a good estimate of the test accuracy.

Next, we use cross-validation to find the $\sigma$ that leads to the best accuracy.
We here also keep track of the standard deviation over $k$ cross-validation folds to get a feeling for how certain we are about the accuracy estimate.

In [ ]:
results = np.zeros(len(widths))
results_std = np.zeros(len(widths))
for i,width in enumerate(widths):
    sg_kernel.set_width(width)
    sg_result = sg_xval.evaluate()
    sg_result = sg.CrossValidationResult.obtain_from_generic(sg_result)
    results[i]=sg_result.mean
    results_std[i]=sg_result.std_dev

best_width = widths[np.argmax(results)]

In [ ]:
plt.plot(np.log2(widths), results, 'b-')
plt.plot(np.log2(widths), results-2*results_std, 'b--')
plt.plot(np.log2(widths), results+2*results_std, 'b--')
plt.xlabel(r"$\log_2 (\sigma)$")
plt.ylabel("X-validation accuracy")
plt.grid(True)

plt.plot(np.log2(widths[np.argmax(results)]), np.max(results), 'r*', markersize=15);

Let's use the found kernel bandwidth and check the test performance.

In [ ]:
sg_kernel.set_width(best_width)
sg_svm.train(sg_X_train);

sg_y_pred = sg_svm.apply(sg_X_test)
plot_two_class_predictions(X_test, sg_y_pred.get_labels())
print "Test accuracy:", sg.AccuracyMeasure().evaluate(sg_y_pred, sg_y_test)

As you can see, the cross-validation estimate is still over-estimating the performance slightly: the test performance is a bit lower.
This is very common and reminds us that we should never report cross-validation accuracy as test performance (i.e. we used the testing data for tuning parameters) -- it will most likely be less on a **real** test set.

Finally, let's have a look atht the support vectors of the final model.
As you can see, only the datapoints close to the decision boundary are part of the SVM model.
This makes the SVM very efficient to apply, as only few of the original datapoints need to be touched to predict the class of a test point.

In [ ]:
plot_support_vectors(sg_svm,X_train,y_train,x_range=[-4,4], y_range=[-4,4])